# PV Degradation Tools Tutorial - 1
### Relative Humidity and Edge Seal Width
***
Matthew Brown

2023.06.02
***

**Requirements:**
- weather file (psm3 preferred) demo is provided

**Objectives:**
1. Read in necessary weather data
2. Generate POA and Solar Position
3. Calculate edge seal width

In [1]:
import os
import numpy as np
import pandas as pd

import pvdeg 
from pvdeg import TEST_DATA_DIR

## 1.
From a psm3 (or TMY3 or EPW) file we need the following:
- DNI
- DHI
- GHI
- Temperature, Dry-Bulb
- Temperature, Dew-Point
- Wind Speed
- Relative Humidity
- Latitude
- Longitude
- altitude

In [2]:
PSM_FILE = os.path.join(TEST_DATA_DIR,'psm3_pytest.csv')
WEATHER, META = pvdeg.weather.read(PSM_FILE,'psm')

dni = WEATHER.dni
dhi = WEATHER.dhi
ghi = WEATHER.ghi

time_range = WEATHER.index

## 2.

Next we need to generate the solar position (azimuth, elevation, etc) and plane of array irradiance (POA). `PVDEG` has a wrapper for quickly using `PVLIB` to generate these figures.

In [3]:
sol_pos = pvdeg.spectral.solar_position(weather_df=WEATHER, meta=META)

poa_df = pvdeg.spectral.poa_irradiance(weather_df=WEATHER, meta=META, solar_position=sol_pos)

# for now, all we need is the POA_GLOBAL, so grab the column from the dataframe
poa_global = poa_df.poa_global

## 3. 
### Design: Edge Seal Width
`pvdeg.design.edge_seal_width` calculates the width [cm] required for several years of water ingress. If you do not specify the number of years, it will default to 25 years.

In [4]:
psat, psat_avg = pvdeg.humidity.psat(WEATHER['temp_air'])

k = pvdeg.design.k(avg_psat=psat_avg)

edge_seal_width = pvdeg.design.edge_seal_width(k, years=20)

print(psat_avg)
print(k)
print(edge_seal_width)

1.3946499092628233
0.00153182106297676
0.6413923384461668


### Alternate Edge Seal Width
If you have the dew point, or wet-bulb temperature, you can calculate the edge seal width directly.

In [6]:
edge_seal_width = pvdeg.design.edge_seal_from_dew_pt(dew_pt_temp=WEATHER['Dew Point'], years=20)
edge_seal_width

0.40245072298649226

### 6. Material Relative Humidity

In [18]:
rh_front_encap = pvdeg.StressFactors.rh_front_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_back_encap = pvdeg.StressFactors.rh_back_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_backsheet = pvdeg.StressFactors.rh_backsheet_from_encap(rh_back_encap=rh_back_encap,
                                                         rh_surface_outside=data['Relative Humidity'])

In [19]:
print(rh_front_encap[17])
print(rh_back_encap[17])
print(rh_backsheet[17])


50.28910527924019
80.45763598088315
81.22381799044157
